In [ ]:
#Importing libraries
import pandas as pd
import matplotlib.pylab as plt
from scipy import stats
import numpy as np
from itertools import combinations, groupby
from collections import Counter
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
from math import sqrt
import seaborn as sb
import itertools
import datetime
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#Load into pandas dataframe
try:
    df = pd.read_excel("Online_retail_Data.xlsx")
    print("Loading Dataset Successful")
except:
    print("Loading Dataset unsuccessful. Please check if file is present in same folder as this program with file name Online_retail_Data.xlsx")

In [ ]:
#Checking count of rows and columns in dataset
rows, columns = df.shape

print("Row count:",rows,"\nColumn count:",columns)

In [ ]:
print("Available features with data type are:\n")
print(df.dtypes)

In [ ]:
#Describing features
df.head()

InvoiceNo - Unique number assigned to a shipment for billing purpose

StockCode - Helps track the item for inventory

Description - Product name

Quantity - Amount purchased

InvoiceDate - Date and time of purchase

UnitPrice - Cost of product

CustomerID - Unique id for customer

Country - Contry of purchase

In [ ]:
#Check for null values in our columns
print(df.isnull().sum())

In [ ]:
#Removing null valued rows
df = df[~((df['CustomerID'].isnull()) | (df['Description'].isnull()))]
print(df.isnull().sum())

In [ ]:
#Removing data with quantity value less than 0
df = df[~(df['Quantity']<=0)]

In [ ]:
#Removing data with nan Descripton values
df = df[~((df['Description'] == 'nan') | (df['Description'] == 'NAN'))]

In [ ]:
#Creating new useful features

#Total cost of purchase
df['total_cost'] = df['Quantity'] * df['UnitPrice']

#Purchase made on hour of day
df['hour'] = pd.DatetimeIndex(df['InvoiceDate']).hour
df['day_of_week'] = pd.DatetimeIndex(df['InvoiceDate']).dayofweek
df['month'] = pd.DatetimeIndex(df['InvoiceDate']).month
df['year'] = pd.DatetimeIndex(df['InvoiceDate']).year

df.head()

In [ ]:
#Product sales grouped by hour of day

check = pd.DataFrame()
check['hour'] = df['hour']
check['count'] = df['InvoiceNo']

display = check.groupby('hour',as_index=False)['count'].count()

#Plotting bar chart
plt.bar(display['hour'],display['count'])
plt.xlabel('Hour of day')
plt.ylabel('Count of item sold')
plt.xticks(display['hour'])
plt.title('Product sales grouped by hour of day')
plt.show()

Looks like highest number of product sales are made in the afternoon 12 p.m. and sales fall after the evening till morning 6 a.m. This may be because people make most purchases in the afternoon when they have free time.

In [ ]:
#Product sales grouped by month

check['month'] = df['month']

display = check.groupby('month',as_index=False)['count'].count()

#Plotting bar chart
plt.bar(display['month'],display['count'])
plt.xlabel('Month')
plt.ylabel('Count of item sold')
plt.xticks(display['month'])
plt.title('Product sales grouped by month(January=1 to December=12)')
plt.show()

As seen above, the highest count of item were sold in the month of November. This may be due to there is some kind of promotional or seasonal sale on the products in month of November.

In [ ]:
#Product sales grouped by country

check['Country'] = df['Country']

display = check.groupby('Country',as_index=False)['count'].count()

#Plotting bar chart
plt.figure(figsize=(12,10))
plt.bar(display['Country'],display['count'])
plt.xlabel('Country')
plt.ylabel('Count of item sold')
plt.xticks(display['Country'], rotation='vertical')
plt.title('Product sales grouped by Country')
plt.show()

From above figure we can infer that ABC company is United Kingdom based company or it has biggest market in UK.

In [ ]:
#Top selling product by average quantity sold

check['Description'] = df['Description']
check['Quantity'] = df['Quantity']

display = check.groupby('Description',as_index=False)['Quantity'].mean()

display.sort_values(by=['Quantity'],ascending=False,inplace=True)


In [ ]:
display.head(10)

In [ ]:
#Top selling product by average revenue earned

#check['Description'] = df['Description']
check['total_cost'] = df['total_cost']

display = check.groupby('Description',as_index=False)['total_cost'].mean()

display.sort_values(by=['total_cost'],ascending=False,inplace=True)

In [ ]:
display.head(10)

In [ ]:
#Converting dataframe to series to perform analysis
orders = df.set_index('InvoiceNo')['StockCode'].rename('item_id')
print(orders.head(10))
type(orders)

In [ ]:
#Lets check out some order statistics
print('Dimension:',orders.shape[0],'\nUnique orders:',len(orders.index.unique()),
      '\nUnique items:',len(orders.value_counts()))

In [ ]:
#Building functions for association rule

#Return count for items and item pairs
def freq(iterable):
    if isinstance(iterable, pd.core.series.Series):
        return iterable.value_counts().rename("freq")
    else:
        # Use a dictionary to count item pairs
        counts = Counter()
        for pair in iterable:
            counts[pair] += 1
        return pd.Series(counts).rename("freq")


#Return count of unique orders
def order_count(order_item):
    return len(set(order_item.index))




#To handle big dataset, I am using generator to give item pairs
def get_item_pairs(order_item):
    order_item = order_item.reset_index().to_numpy()
    for order_id, order_object in groupby(order_item, lambda x: x[0]):
        item_list = [item[1] for item in order_object]
              
        for item_pair in combinations(item_list, 2):
            yield tuple(item_pair)  # Ensure item_pair is a tuple of strings (StockCode)



# Returns frequency and support associated with item
def merge_item_stats(item_pairs, item_stats):
    return (item_pairs
                .merge(item_stats.rename(columns={'freq': 'freqA', 'support': 'supportA'}),
                       left_on='item_A', right_index=True)
                .merge(item_stats.rename(columns={'freq': 'freqB', 'support': 'supportB'}),
                       left_on='item_B', right_index=True))


# Returns name associated with item
def merge_item_name(rules, item_name):
    columns = ['itemA','itemB','freqAB','supportAB','freqA','supportA','freqB','supportB', 
               'confidenceAtoB','confidenceBtoA','lift']
    rules = (rules
                .merge(item_name.rename(columns={'item_name': 'itemA'}), left_on='item_A', right_on='item_id')
                .merge(item_name.rename(columns={'item_name': 'itemB'}), left_on='item_B', right_on='item_id'))
    return rules[columns]

In [ ]:
#Function for trigerring the rule mining

def association_rules(order_item, min_support):

    print("Starting order_item: {:22d}".format(len(order_item)))


    # Calculate item frequency and support
    item_stats             = freq(order_item).to_frame("freq")
    item_stats['support']  = item_stats['freq'] / order_count(order_item) * 100


    # Filter from order_item items below min support 
    qualifying_items       = item_stats[item_stats['support'] >= min_support].index
    order_item             = order_item[order_item.isin(qualifying_items)]

    print("Items with support >= {}: {:15d}".format(min_support, len(qualifying_items)))
    print("Remaining order_item: {:21d}".format(len(order_item)))


    # Filter from order_item orders with less than 2 items
    order_size             = freq(order_item.index)
    qualifying_orders      = order_size[order_size >= 2].index
    order_item             = order_item[order_item.index.isin(qualifying_orders)]

    print("Remaining orders with 2+ items: {:11d}".format(len(qualifying_orders)))
    print("Remaining order_item: {:21d}".format(len(order_item)))


    # Recalculate item frequency and support
    item_stats             = freq(order_item).to_frame("freq")
    item_stats['support']  = item_stats['freq'] / order_count(order_item) * 100


    # Get item pairs generator
    item_pair_gen          = get_item_pairs(order_item)


    # Calculate item pair frequency and support
    item_pairs              = freq(item_pair_gen).to_frame("freqAB")
    item_pairs['supportAB'] = item_pairs['freqAB'] / len(qualifying_orders) * 100

    print("Item pairs: {:31d}".format(len(item_pairs)))


    # Filter from item_pairs those below min support
    item_pairs              = item_pairs[item_pairs['supportAB'] >= min_support]

    print("Item pairs with support >= {}: {:10d}\n".format(min_support, len(item_pairs)))


    # Create table of association rules and compute relevant metrics
    item_pairs = item_pairs.reset_index().rename(columns={'level_0': 'item_A', 'level_1': 'item_B'})
    item_pairs = merge_item_stats(item_pairs, item_stats)
    
    item_pairs['confidenceAtoB'] = item_pairs['supportAB'] / item_pairs['supportA']
    item_pairs['confidenceBtoA'] = item_pairs['supportAB'] / item_pairs['supportB']
    item_pairs['lift']           = item_pairs['supportAB'] / (item_pairs['supportA'] * item_pairs['supportB'])
    
    
    # Return association rules sorted by lift in descending order
    return item_pairs.sort_values('lift', ascending=False)

In [ ]:
#Running the model
rules = association_rules(orders,0.01)

In [ ]:
#lets display the results

#Replacing item id with item name
item_name = pd.DataFrame()
item_name['item_id'] = df['StockCode']
item_name['item_name'] = df['Description']

item_name.drop_duplicates(inplace=True)

#replacing item id with item names
final_result = merge_item_name(rules,item_name).sort_values('lift',ascending=False)

#Filtering results based on lift value
final_result = final_result[(final_result['lift'] > 1)]

In [ ]:
#Function for getting the results

def item_bought_together(key):
    return final_result[final_result['itemA']==key].head(10)

In [ ]:
#Change the key that is item name to see items bought together

result = item_bought_together('ENVELOPE 50 ROMANTIC IMAGES')
result

In [ ]:
# Ensure 'InvoiceDate' is in datetime format
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# Adding week column to dataset
df['week'] = df['InvoiceDate'].dt.isocalendar().week

# Display the first few rows to verify
print(df.head())


In [ ]:
#We will first filter our data to keep only top 10 products

#Top selling product by quantity of product sold

check['Description'] = df['Description']
check['Quantity'] = df['Quantity']
check['StockCode'] = df['StockCode']
check['week'] = df['week']

display = check.groupby(['Description'],as_index=False).agg({'Quantity':sum, 'StockCode':"count"})
display.sort_values(by=['Quantity'],ascending=False,inplace=True)

In [ ]:
#Top selling product by quantity of product sold and frequency of purchase
display.sort_values(by=['StockCode'],ascending=False,inplace=True)

In [ ]:
display['Description'].head(10)

In [ ]:
df = df[(df['Description'] == 'WHITE HANGING HEART T-LIGHT HOLDER') | 
       (df['Description'] == 'REGENCY CAKESTAND 3 TIER') |
       (df['Description'] == 'JUMBO BAG RED RETROSPOT') |
       (df['Description'] == 'ASSORTED COLOUR BIRD ORNAMENT') |
       (df['Description'] == 'PARTY BUNTING') |
       (df['Description'] == 'LUNCH BAG RED RETROSPOT') |
       (df['Description'] == 'SET OF 3 CAKE TINS PANTRY DESIGN ') |
       (df['Description'] == 'LUNCH BAG  BLACK SKULL.') |
       (df['Description'] == 'POSTAGE') |
       (df['Description'] == 'PACK OF 72 RETROSPOT CAKE CASES')]

In [ ]:
#creating new data frame with only required features
ts  = pd.DataFrame()
ts['Description'] = df['Description']
ts['Quantity'] = df['Quantity']
ts['InvoiceDate'] = df['InvoiceDate'].dt.date
ts['InvoiceDate'] = pd.to_datetime(ts['InvoiceDate'])
ts['month'] = df['month']
ts['week'] = df['week']


In [ ]:
#For first product
#product = 'LUNCH BAG RED RETROSPOT'
def building_data(product):
    
    p1 = pd.DataFrame()
    p1['InvoiceDate'] = ts[ts['Description'] == product].InvoiceDate
    p1['Quantity'] = ts[ts['Description'] == product].Quantity
    p1['month'] = ts[ts['Description'] == product].month
    p1['week'] = ts[ts['Description'] == product].week

    p1 = p1.groupby('InvoiceDate')['Quantity'].sum().reset_index()


    #setting invoice date as index
    indexed_p1 = p1.set_index(['InvoiceDate'])

    #Resampling our data to weekly frequency and dropping null
    y = indexed_p1['Quantity'].resample('W').mean()
    y = y.dropna()
    
    #Checking size for train test split
    size = y.shape[0]
    
    print('data built')
    return y

In [ ]:
def plot_data(product_to_plot):
    
    #Plotting product quantity sold weekly
    product_to_plot.plot(figsize=(15,6))
    plt.xlabel("Date sampled weekly")
    plt.ylabel("Average quantity sold")
    plt.title("Quantity of product sold weekly")
    plt.show()


    #Performing time-series decomposition to check trend,seasonility and noise

    from pylab import rcParams
    rcParams['figure.figsize'] = 18,8

    decomposition = sm.tsa.seasonal_decompose(product_to_plot, freq=7, model='additive')
    fig= decomposition.plot()
    plt.show()


In [ ]:
def sarimax(product_data,size):
    # generating parameter combinations
    #final_results = []
    
    #Train and test splitting
    train_size = round(size * (80/100))
    #print('Train size:',train_size)
    test_size = (size - train_size)+1
    #print('Test size:',test_size)

    train = pd.DataFrame()
    test = pd.DataFrame()

    train = product_data.head(train_size)
    test = product_data.tail(test_size)
    

    p=d=q=range(0,2)
    pdq = list(itertools.product(p,d,q))
    seasonal_pdq = [(x[0],x[1],x[2],6)
    for x in list(itertools.product(p,d,q))]
    # using param combinations to find best RMSE
    RMSE_list = pd.DataFrame({}, columns=['param','param_seasonal','RMSE'])
    for param in pdq:
        for param_seasonal in seasonal_pdq:
            try:
            #Running the model
                mod = sm.tsa.statespace.SARIMAX(train,order=param,seasonal_order = param_seasonal,enforce_stationarity=False,enforce_invertibility=False)
                results = mod.fit()
                output = results.predict(start=train_size+1,end=train_size+test_size)
                #Calculating error for different model parameters
                rmse = sqrt(mean_squared_error(test, output))
                #print(rmse)
                temp = pd.DataFrame([[param,param_seasonal,rmse]], columns=['param','param_seasonal', 'RMSE'])
                RMSE_list = RMSE_list.append(temp, ignore_index=True)
                del temp
            except:
                continue
    m = np.amin(RMSE_list['RMSE'].values) # finding the lowest error value
    l = RMSE_list['RMSE'].tolist().index(m) # extracting the index of lowest error value
    minimum_rmse = RMSE_list.iloc[l,:] # Storing the final selected model parameters with lowest RMSE
    #print('Lowest rmse:',minimum_rmse)
    # Running the model with finalized parameters
    print('Predicting weekly quantity required.....\n')
    mod = sm.tsa.statespace.SARIMAX(product_data,order=minimum_rmse['param'],seasonal_order=minimum_rmse['param_seasonal'],enforce_stationarity=False,enforce_invertibility=False)
    results = mod.fit()
    final_p = results.predict(start=size+1,end=size+12)
    final_p[final_p<0] = 0
    print(final_p)
    print('-------------------------------------------------------------------------------')
    return final_p

In [ ]:
#Checking if we only have the data of top 10 products
input_list = df['Description'].unique()

In [ ]:
for i in input_list:
    print('Performing prediction for: ',i)
    prod = building_data(i)
    print('Implementing model....')
    sarimax(prod,size).head(12)

In [ ]:
def sarimax(product_data, size):
    # Train and test splitting
    train_size = round(size * (80 / 100))
    test_size = size - train_size + 1

    train = product_data.head(train_size)
    test = product_data.tail(test_size)

    # Generating parameter combinations
    p = d = q = range(0, 2)
    pdq = list(itertools.product(p, d, q))
    seasonal_pdq = [(x[0], x[1], x[2], 6) for x in list(itertools.product(p, d, q))]

    # Using parameter combinations to find best RMSE
    RMSE_list = pd.DataFrame({}, columns=['param', 'param_seasonal', 'RMSE'])
    for param in pdq:
        for param_seasonal in seasonal_pdq:
            try:
                mod = sm.tsa.statespace.SARIMAX(train,
                                                order=param,
                                                seasonal_order=param_seasonal,
                                                enforce_stationarity=False,
                                                enforce_invertibility=False)
                results = mod.fit()
                output = results.predict(start=train_size + 1, end=train_size + test_size)
                rmse = sqrt(mean_squared_error(test, output))
                temp = pd.DataFrame([[param, param_seasonal, rmse]], columns=['param', 'param_seasonal', 'RMSE'])
                RMSE_list = RMSE_list.append(temp, ignore_index=True)
                del temp
            except:
                continue

    # Selecting model parameters with lowest RMSE
    m = np.amin(RMSE_list['RMSE'].values)
    l = RMSE_list['RMSE'].tolist().index(m)
    minimum_rmse = RMSE_list.iloc[l, :]

    # Running the model with finalized parameters
    print('Predicting weekly quantity required.....\n')
    mod = sm.tsa.statespace.SARIMAX(product_data,
                                    order=minimum_rmse['param'],
                                    seasonal_order=minimum_rmse['param_seasonal'],
                                    enforce_stationarity=False,
                                    enforce_invertibility=False)
    results = mod.fit()
    final_p = results.predict(start=size + 1, end=size + 12)
    final_p[final_p < 0] = 0
    print(final_p)
    print('-------------------------------------------------------------------------------')
    return final_p

In [ ]:
#Checking if we only have the data of top 10 products
input_list = df['Description'].unique()

In [ ]:
# Assuming input_list is already defined
for product in input_list:
    print('Performing prediction for:', product)
    product_data = building_data(product)  # Assuming you have a function building_data to get data for each product
    print('Implementing model....')
    size = len(product_data)  # Determine appropriate size for SARIMAX model
    sarimax(product_data, size).head(12)  # Adjust as needed for output and display


In [ ]:
#Get data plotted for specific product, Just key in the value for input_list
plot_this = building_data(input_list[1])
print('Plotting data for :',input_list[1])
plot_data(plot_this)